In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Setting the file path

In [ ]:
import os
os.chdir("/content/drive/MyDrive/drive/Supplemental Material/TAVER-Entity-Resolution-main/TAVER-entity resolution")

In [ ]:
!pip install tensorflow-gpu==1.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Loading data and importing library files

In [ ]:
import pandas as pd
import numpy as np
import string
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")  
import os
import random
import tensorflow as tf
import torch
from sentence_transformers import util

In [ ]:
file_path = 'data/Structured/DBLP-ACM/'

table_a_df = pd.read_csv(file_path + 'tableA.csv')
table_b_df = pd.read_csv(file_path + 'tableB.csv')
train_df = pd.read_csv(file_path + 'train.csv')
valid_df = pd.read_csv(file_path + 'valid.csv')
test_df = pd.read_csv(file_path + 'test.csv')

table_a_df.shape, table_b_df.shape, train_df.shape, test_df.shape

((2616, 5), (2294, 5), (7417, 3), (2473, 3))

# 2 Building unsupervised time representation models and matching models

In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.contrib import slim
from tensorflow.contrib import layers as tflayers
from torch.autograd import Variable
import torchvision
import torch.utils.data as Data
import torch

from sklearn import metrics

In [ ]:
global max_batch_n, batch_index
global LR_PRIMAL, LR_DUAL, X_dim, h_dim, z_dim, eps_dim, eps_nbasis
global encoder, decoder, discriminator

## 2.1 Defining the required functions

In [ ]:
def TupleEmbedding(all_sentences, sentence_embedding_model):
    '''
    all_sentences: 包含所有元组句子的列表
    sentence_embedding_model: 将句子转换为句子向量的模型
    founction：将所有的句子，转换为对应的sentence vector
    '''
    sentences_vector = []
    for sen in all_sentences:
        sentences_vector.append(sentence_embedding_model.encode(sen))
    return np.array(sentences_vector)

def GetBatchData(train_loader):
    '''
    传入torch的dataloader,然后生成对应的分批数据
    '''
    global batch_index
    for step, (batch_x, batch_y) in enumerate(train_loader):
        if step != batch_index:
            continue
        
        sample_x = batch_x.view(BATCH_SIZE, -1).data.numpy()
    return sample_x


def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def get_zlogprob(z):
    # temp = tf.clip_by_value(predict_1, 1e-8, 1.0)
    logprob = -0.5 * tf.reduce_sum(z*z  + np.log(2*np.pi), [1])
    return logprob

def bit_product_sum(x, y):
    return sum([item[0] * item[1] for item in zip(x, y)])

def get_reconstr_err(de_x, x):
    x_multi = tf.multiply(x, x)
    de_x_multi = tf.multiply(de_x, de_x)
    x_de_x_multi = tf.multiply(x, de_x)

    x_mul_sum = tf.reduce_sum(x_multi)
    de_x_mul_sum = tf.reduce_sum(de_x_multi)
    x_de_mul_sum = tf.reduce_sum(x_de_x_multi)

    cosine = tf.divide(x_de_mul_sum, tf.add(x_mul_sum, de_x_mul_sum))
    reconst_err = tf.subtract(2., cosine)

    return reconst_err

## 2.2 Building encoders, decoders and adversarial networks (discriminator)

In [ ]:
def encoder_func(x):
    size = x.shape[0]
    # eps = tf.random_normal(tf.stack([eps_nbasis, size, eps_dim]))

    net = slim.fully_connected(x, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    # net = slim.dropout(net, 0.3, is_training=False)
    # net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)

    z_mean = slim.fully_connected(net, z_dim, activation_fn=None)
    z_logstd = slim.fully_connected(net, z_dim, activation_fn=None)
    # logstd = log(std)
    return z_mean, z_logstd

def decoder_func(z):
    net = z
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    xlogits = slim.fully_connected(net, X_dim, activation_fn=None)
    return xlogits

def discriminator_func(x, z):
    # Theta
    with tf.variable_scope("theta"):
        fc_argscope = slim.arg_scope([slim.fully_connected], activation_fn=lrelu)
        with fc_argscope:
            net = slim.fully_connected(x, 1024, scope='fc_0')
            net = slim.fully_connected(net, 1024, scope='fc_1')
            net = slim.fully_connected(net, 1024, scope='fc_2')
        theta = slim.fully_connected(net, 8192, activation_fn=tf.nn.elu, scope='theta',
                  weights_initializer=tf.truncated_normal_initializer(stddev=1e-5))

    with tf.variable_scope("s"):
        fc_argscope = slim.arg_scope([slim.fully_connected],
          activation_fn=lrelu)
        with fc_argscope:
            net = slim.fully_connected(z, 1024, scope='fc_0')
            net = slim.fully_connected(net, 1024, scope='fc_1')
            net = slim.fully_connected(net, 1024, scope='fc_2')
        s = slim.fully_connected(net, 8192, activation_fn=None, scope='s')

    with tf.variable_scope("xonly"):
        fc_argscope = slim.arg_scope([slim.fully_connected],
          activation_fn=lrelu)
        with fc_argscope:
            net = slim.fully_connected(x, 1024, scope='fc_0')
            net = slim.fully_connected(net, 1024, scope='fc_1')
            net = slim.fully_connected(net, 1024, scope='fc_2')
        Tx = slim.fully_connected(net, 1, activation_fn=None, scope='Tx',
              weights_initializer=tf.truncated_normal_initializer(stddev=1e-5))

    with tf.variable_scope("zonly"):
        fc_argscope = slim.arg_scope([slim.fully_connected],
          activation_fn=lrelu)
        with fc_argscope:
            net = slim.fully_connected(z, 1024, scope='fc_0')
            net = slim.fully_connected(net, 1024, scope='fc_1')
            net = slim.fully_connected(net, 1024, scope='fc_2')
        Tz = slim.fully_connected(net, 1, activation_fn=None, scope='Tz',
              weights_initializer=tf.truncated_normal_initializer(stddev=1e-5))

    T = tf.reduce_sum(theta * s, [1], keep_dims=True) + Tx + Tz
    T = tf.squeeze(T, 1)
    T += 0.5 * tf.reduce_sum(tf.square(z), [1])
    return T


## 2.3 Constructing computational maps for entity representation model training

In [ ]:
def BuildComputationalGraph(x_real):
    '''
    构建计算图，输出两个损失值，和对应的参数
    '''
    z_sampled = tf.random_normal([BATCH_SIZE, z_dim])
    eps = tf.random_normal([BATCH_SIZE, z_dim])

    mean, logstd = encoder(x_real)
    std = tf.exp(logstd)
    z_real = mean + eps*std

    z_mean = tf.stop_gradient(mean)
    var = tf.multiply(std, std)
    z_var = tf.stop_gradient(var)
    z_std = tf.stop_gradient(std)

    z_norm = (z_real - z_mean) / z_std
    Td = discriminator(x_real, z_norm) 
    Ti = discriminator(x_real, z_sampled) 
    logz = -0.5 * tf.reduce_sum(z_real*z_real  + np.log(2*np.pi), [1])

    z_var = tf.clip_by_value(z_var, 1e-8, 1.0)
    logr = -0.5 * tf.reduce_sum(z_norm*z_norm + tf.log(z_var) + np.log(2*np.pi), [1])

    decoder_out = decoder(z_real)  

    c_dim = 1
    beta = 1
    factor = 10.0 / (X_dim * c_dim)

    # # Primal loss
    reconst_err = get_reconstr_err(decoder_out, x_real)
    KL = Td + logr - logz
    ELBO = reconst_err + KL
    loss_primal = factor * tf.reduce_mean(3*reconst_err + beta*KL)

    # Mean values
    ELBO_mean = tf.reduce_mean(ELBO)
    KL_mean = tf.reduce_mean(KL)
    reconst_err_mean = tf.reduce_mean(reconst_err)

    # Dual loss
    d_loss_d = tf.reduce_mean(
      tf.nn.sigmoid_cross_entropy_with_logits(logits=Td, labels=tf.ones_like(Td)))
    d_loss_i = tf.reduce_mean(
      tf.nn.sigmoid_cross_entropy_with_logits(logits=Ti, labels=tf.zeros_like(Ti)))
    loss_dual = d_loss_i + d_loss_d
    
    qvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "encoder")
    pvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "decoder")
    dvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "discriminator")

    td = tf.reduce_mean(Td)

    return loss_primal, loss_dual, pvars+qvars, dvars, ELBO_mean, reconst_err_mean, KL_mean,td

def GetTrainOp(loss_primal, loss_dual, vars_primal, vars_dual):
    learning_rate = LR_PRIMAL
    learning_rate_adversary = LR_DUAL

    # Train step
    primal_optimizer = tf.train.AdamOptimizer(learning_rate, use_locking=True, beta1=0.5)
    adversary_optimizer = tf.train.AdamOptimizer(learning_rate_adversary, use_locking=True, beta1=0.5)

    primal_grads = primal_optimizer.compute_gradients(loss_primal, var_list=vars_primal)
    adversary_grads = adversary_optimizer.compute_gradients(loss_dual, var_list=vars_dual)

    primal_grads = [(grad, var) for grad, var in primal_grads if grad is not None]
    adversary_grads = [(grad, var) for grad, var in adversary_grads if grad is not None]

    allgrads = [grad for grad, var in primal_grads + adversary_grads]
    with tf.control_dependencies(allgrads):
        primal_train_step = primal_optimizer.apply_gradients(primal_grads)
        adversary_train_step = adversary_optimizer.apply_gradients(adversary_grads)

    train_op = tf.group(primal_train_step, adversary_train_step)

    return train_op

## 2.4 Define the required functions and read the embedding vector

In [ ]:
def Get2_WD(a_e, a_v, b_e, b_v):
    '''
    传入批量的两个多元高斯分布的均值和方差,(batch_size, z_dim)
    计算这一批多元高斯分布的平均距离,根据是否匹配，分别计算距离
    和对应的距离向量
    '''
    ab_e = tf.subtract(a_e, b_e)
    ab_v = tf.subtract(a_v, b_v)

    ab_e2 = tf.multiply(ab_e, ab_e)
    ab_v2 = tf.multiply(ab_v, ab_v)

    e_v = tf.add(ab_e2, ab_v2)
    e_v_sum = tf.reduce_mean(e_v, 1)

    return e_v_sum, e_v

def ClassifyModel(x):
    net = x
    h_dim = 300
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.dropout(net, 0.3, is_training=False)

    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.dropout(net, 0.3, is_training=False)

    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.dropout(net, 0.3, is_training=False)

    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)

    prediction = slim.fully_connected(net, 2, activation_fn=tf.nn.softmax)
    return prediction

classify_model = tf.make_template('ClassifyModel', ClassifyModel)



In [ ]:
X = []
with open(file_path + 'X_array.txt', 'r') as fp:
    temp = fp.readlines()
for item in temp:
    s = item.strip('\n')
    slist = s.split()
    slist = [eval(i) for i in slist]
    X.append(slist)
X = np.array(X)
X = torch.FloatTensor(X).data.numpy()
print(X.shape)
sentences_vector = X

(4910, 768)


In [ ]:
a_id_list = []
b_id_list = []
for i in range(train_df.shape[0]):
    a_id_list.append(train_df.loc[i, 'ltable_id'])
    b_id_list.append(train_df.loc[i, 'rtable_id'])
for i in range(test_df.shape[0]):
    a_id_list.append(test_df.loc[i, 'ltable_id'])
    b_id_list.append(test_df.loc[i, 'rtable_id'])

    
a_id_list = list(set(a_id_list))
b_id_list = list(set(b_id_list))
    
X_train_test = []
for a_id in a_id_list:
    X_train_test.append(sentences_vector[a_id])
for b_id in b_id_list:
    X_train_test.append(sentences_vector[table_a_df.shape[0] + b_id])

X_train_test = np.array(X_train_test)
X_train_test = torch.FloatTensor(X_train_test).data.numpy()
print(X_train_test.shape)

(4443, 768)


In [ ]:
## For large data sets, eg:DBLP-ACM, DBLP-Scholar, iTunes-Amazon
avb_train = X_train_test 
## For small data sets
# avb_train = X

In [ ]:
BATCH_SIZE = 20
Y_fake = torch.linspace(1, 100, avb_train.shape[0])
torch_dataset = Data.TensorDataset(torch.FloatTensor(avb_train), Y_fake)

train_loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

## 定义超参数
X_dim = avb_train.shape[1]
h_dim = 300
z_dim = 128
eps_dim = 32
eps_nbasis = 16

# 获取模型变量
encoder = tf.make_template('encoder', encoder_func)
decoder = tf.make_template('decoder', decoder_func)
discriminator = tf.make_template('discriminator', discriminator_func)

# 构建计算图
LR_PRIMAL = 1e-5
LR_DUAL = 2e-5

x_real = tf.placeholder(tf.float32, shape=[BATCH_SIZE, X_dim])
loss_primal, loss_dual, vars_primal, vars_dual, ELBO_mean, reconst_err_mean, KL_mean,td = BuildComputationalGraph(x_real)
train_op1 = GetTrainOp(loss_primal, loss_dual, vars_primal, vars_dual)

In [ ]:
sentences_vector = X
pairs_0, pairs_1 = [], []
pairs_0.extend(train_df[train_df['label'] == 0].values.tolist())
pairs_1.extend(train_df[train_df['label'] == 1].values.tolist())

In [ ]:
global id_0, id_1, num_0, num_1, len_0, len_1
id_0, id_1 = 0, 0
id_all = 0
num_0, num_1 = 15, 5
total_num01 = num_0 + num_1
len_0, len_1 = len(pairs_0), len(pairs_1)
# len_all = len(pairs_all)


def GetBatchData_2(sentences_vector):
    '''
    新的批数据生产函数
    '''
    global id_0, id_1, num_0, num_1, len_0, len_1, len_all, id_all
    
#     len_0 = len_0 * 1 // 6
#     len_1 = len_1 * 1 // 6
    
    x_a, x_b, y = [], [], []

 
    for i in range(num_0):
        id_0 = (id_0 + 1) % len_0
        x_a.append(sentences_vector[int(pairs_0[id_0][0])])
        x_b.append(sentences_vector[int(table_a_df.shape[0] + pairs_0[id_0][1])])
        y.append(0)
        
    for i in range(num_1):
        id_1 = (id_1 + 1) % len_1
        x_a.append(sentences_vector[int(pairs_1[id_1][0])])
        x_b.append(sentences_vector[int(table_a_df.shape[0] + pairs_1[id_1][1])])
        y.append(1)
    
    x_a = np.array(x_a).reshape(total_num01, -1)
    x_b = np.array(x_b).reshape(total_num01, -1)
    y = np.array(y).reshape(total_num01, -1)
    
    return x_a, x_b, y

## 2.5 Constructing computational maps for binary classification model (matching model) training

In [ ]:
def TrainClassifyModel(x_a, x_b, y):
    # Encoder
    ez_a, logstd_a = encoder(x_a)
    ez_b, logstd_b = encoder(x_b)
    
    vz_a = tf.exp(logstd_a)
    vz_b = tf.exp(logstd_b)

    # 2-WassersteinDistance, (batch_size, 1), (batch_size, z_dim)
    WD_disc, WD_vector = Get2_WD(ez_a, vz_a, ez_b, vz_b)

    prediction = classify_model(WD_vector)
    predict_0 = tf.slice(prediction, [0,0],[-1,1])
    predict_1 = tf.slice(prediction, [0,1],[-1,1])

    
    M = 0.5  
    class_w = 3.0  
    
    predict_1 = tf.clip_by_value(predict_1, 1e-8, 1.0)
    predict_0 = tf.clip_by_value(predict_0, 1e-8, 1.0)
    loss_1 = -tf.add(tf.multiply(class_w*y, tf.log(predict_1)), tf.multiply(1*(1.0-y), tf.log(predict_0)))
    loss_2 = tf.add(tf.multiply(class_w*y, WD_disc), tf.multiply(1.0-y, tf.maximum(0.0,M-WD_disc)))

    loss_1 = tf.reduce_mean(loss_1)
    loss_2 = tf.reduce_mean(loss_2)
    total_loss = loss_1 + loss_2

    
    encoder_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "encoder")
    classify_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "ClassifyModel")

    train_op2 = GetTrainOp(2*loss_1, 1*loss_2, encoder_vars+classify_vars, encoder_vars)

    return train_op2, loss_1, loss_2

LR_PRIMAL = 1e-5
LR_DUAL = 2e-5

x_a = tf.placeholder(tf.float32, shape=[total_num01, X_dim])
x_b = tf.placeholder(tf.float32, shape=[total_num01, X_dim])
y = tf.placeholder(tf.float32, shape=[total_num01, 1])
train_op2, loss_1, loss_2 = TrainClassifyModel(x_a, x_b, y)

# 3 Model Training

## 3.1 Training unsupervised entity representation models

In [ ]:
sess = tf.InteractiveSession()  
sess.run(tf.global_variables_initializer())  

batch_index = 0
max_batch_n = avb_train.shape[0] // BATCH_SIZE
progress = tqdm_notebook(range(60*max_batch_n))
for i in progress:
    x = GetBatchData(train_loader)
    batch_index = (batch_index + 1) % max_batch_n

    # sess.run(train_op1)
    _,loss_p,loss_d, r_e,kl,td1 = sess.run([train_op1,loss_primal,loss_dual,reconst_err_mean,KL_mean,td], 
                                           feed_dict={x_real:x})
    progress.set_description('loss_primal= %.4f, loss_dual= %.4f, re=%.4f - ' % (loss_p,loss_d,r_e))

  0%|          | 0/13320 [00:00<?, ?it/s]

*      loss_primal= 0.0853, loss_dual= 2.0841, re=1.8313

## 3.2 Training matching models

In [ ]:
sentences_vector = X
X_a = []
X_b = []
for i in range(test_df.shape[0]):
    a_id = test_df.loc[i, 'ltable_id']
    b_id = test_df.loc[i, 'rtable_id']
    X_a.append(sentences_vector[a_id])
    X_b.append(sentences_vector[table_a_df.shape[0] + b_id])
X_a = np.array(X_a)
X_b = np.array(X_b)

Y = test_df['label'].values

In [ ]:
F1,P,R = 0,0,0

In [ ]:
# 训练过程
max_batch_n_2 = train_df.shape[0] // total_num01
epoch_n = 20000 * max_batch_n_2
progress = tqdm_notebook(range(epoch_n))

LR_PRIMAL = 1e-5
LR_DUAL = 2e-5

for i in progress:
    Xa, Xb, Yy = GetBatchData_2(sentences_vector)

    _, loss1, loss2 = sess.run([train_op2, loss_1, loss_2], feed_dict={x_a:Xa, x_b:Xb, y:Yy})
    progress.set_description('loss1 = %.5f, loss2 = %.4f - '%(loss1,loss2))

    num_0 = random.randint(8,total_num01)
    num_1 = total_num01 - num_0      

    if (i < max_batch_n_2 * 20000):
        temp = 5*max_batch_n_2
    else:
        temp = max_batch_n_2
#     temp = max_batch_n_2
    if i % temp == 0:
        e_a, v_a = encoder(X_a)
        e_b, v_b = encoder(X_b)
        
        v_a = tf.exp(v_a)
        v_b = tf.exp(v_b)
        
        wd_disc, wd_vector = Get2_WD(e_a, v_a, e_b, v_b)

        # sess.run(wd_vector)
        output = classify_model(wd_vector)
        # output = tf.argmax(output, 1)
        output = sess.run(output)
        out = np.argmax(output, 1)

        precision = metrics.precision_score(Y, out)
        recall = metrics.recall_score(Y, out)
        f1_score = metrics.f1_score(Y, out)
        
        if f1_score > F1:
            F1,P,R = f1_score, precision, recall
            print('epoch = %d, precision = %.3f, recall = %.3f, f1_score = %.4f'%(i,precision,recall,f1_score))


  0%|          | 0/7400000 [00:00<?, ?it/s]

epoch = 0, precision = 0.319, recall = 0.982, f1_score = 0.4820
epoch = 1850, precision = 0.964, recall = 0.966, f1_score = 0.9651
epoch = 5550, precision = 0.986, recall = 0.955, f1_score = 0.9703
epoch = 11100, precision = 0.963, recall = 0.984, f1_score = 0.9733
epoch = 12950, precision = 0.965, recall = 0.986, f1_score = 0.9755
epoch = 16650, precision = 0.969, recall = 0.984, f1_score = 0.9765
epoch = 24050, precision = 0.989, recall = 0.973, f1_score = 0.9807
epoch = 37000, precision = 0.995, recall = 0.971, f1_score = 0.9829
epoch = 88800, precision = 0.989, recall = 0.982, f1_score = 0.9853
epoch = 127650, precision = 0.980, recall = 0.991, f1_score = 0.9854
epoch = 186850, precision = 0.993, recall = 0.980, f1_score = 0.9864
epoch = 223850, precision = 0.987, recall = 0.991, f1_score = 0.9888


KeyboardInterrupt: ignored

*  precision = 0.991, recall = 0.975, f1_score = 0.9830
*  precision = 0.980, recall = 0.986, f1_score = 0.9832
*  precision = 0.984, recall = 0.984, f1_score = 0.9842